# A comparitive study of various most prominent alternatives for Speech to text 
  - Confined to batch data. 
  - Evaluation on the Word Error rate on the cleaned audio files from [Librispeech](http://www.openslr.org/12/)
  -  Aternatives comapred:
    - Mozilla DeepSpeech
    - Google Speech-to-Text
    - Microsoft Azure Speech Cloud Service 

Step 0 - Defining the various functions to:
  - to converting flac files to wav
  - mount google drive 
  - reading the information from the labels
  - reading the wav files to get rate and buffer

In [ ]:
# Packages required

# !pip install deepspeech
# !pip install google-cloud-speech
#!pip install jiwer

In [2]:
# Importing and installing packages

import subprocess
import re
import os
import pathlib
import shutil
import glob

import pandas as pd
import codecs
import unicodedata

from typing import Tuple

from numpy import floor
import time

import wave
import numpy as np
import pandas as pd 
from jiwer import wer


!sudo apt -qq install -y sox


#--------------------------------------------------------
#FYI - 
# The librispeech file corpus comes in various subfolders and you may want to 
# move them to one folder. 
#Cmd for moving all files from all subfolders to the main folder
# for /r %d in (*) do copy "%d" "D:\All Essec\CRP\DeepSpeech\LibriSpeech\All" 
#--------------------------------------------------------


sox is already the newest version (14.4.1-5+deb9u2).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [ ]:
# function to convert all flac file in a directory to wav 

# defining function to replace in a string from left
def rreplace(pattern, sub, string):
    """
    Replaces 'pattern' in 'string' with 'sub' if 'pattern' ends 'string'.
    """
    return re.sub('%s$' % pattern, sub, string)



# defining function to convert flac to wav
def flac_to_wav(inp, dest):
    
   
   #inp - directory to the folder where the flac files are stored
   #dest -directory to the folder where the flac files will be moved

   #note - Dest will be deleted eventually
  print(inp)
  for file in os.listdir(inp):
      if file.endswith('.flac'):
        current_file = os.path.join(inp, file)
        out = rreplace('flac','wav',current_file)
        subprocess.run(["sox", current_file, "-c 1", "-r 16000", out])
#         print('File Name:', current_file)


  print('All files converted done')

  # move all the flac files to a new folder
  source = pathlib.Path(inp)
  dest1 = pathlib.Path(dest)
  dest1.mkdir(exist_ok=True)

  for f in source.glob("*.flac"):
      shutil.move(f, dest1.joinpath(f.name))
#       print(f, 'moved')

  print('All files moved to trash folder')
  # # delete the folder created with the redundant flac files
  shutil.rmtree(dest1)
  print('trash folder deleted successfully')

In [ ]:
# example on how to run flac_to_wav:

inp2 = "/home/jupyter/CRP/Data/audio/Cleaned Audio files"
dest2 = "/home/jupyter/CRP/Data/audio/Cleaned Audio files/toDelete"
flac_to_wav(inp2, dest2)

In [ ]:
# read all label files and make a dataframe
# gives output as a dataframe

def read_from_label(path):
  
  trans = []
  file_name = []

  for file in os.listdir(path):
      current_file = os.path.join(path, file)

      with codecs.open(current_file, "r", "utf-8") as fin:
        for line in fin:
          # Parse each segment line
          first_space = line.find(" ")
          seqid, transcript = line[:first_space], line[first_space+1:]

          #----------------Optional -------------------------
          # We need to do the encode-decode dance here because encode
          # returns a bytes() object on Python 3, and text_to_char_array
          # expects a string.
          # transcript = unicodedata.normalize("NFKD", transcript)  \
          #                         .encode("ascii", "ignore")      \
          #                         .decode("ascii", "ignore")
          #-----------------------------------------------------------
          transcript = transcript.lower().strip()

          trans.append(transcript)
          file_name.append(seqid)
          
        label  = pd.DataFrame(data=trans, columns=[ "transcript"])
        files = pd.DataFrame(data=file_name, columns=["file"])

  labelled_data = label.join(files)
  return labelled_data

In [ ]:
# Example on how to create a data frame from the labelled translation
path = "/home/jupyter/CRP/Data/audio/translation"
labelled = read_from_label(path)
labelled.to_csv('labelled.csv', encoding='utf-8', index=False)

In [ ]:
# function to read the wav files to get rate and buffer

def read_wav_file(filename) -> Tuple[bytes, int]:
    with wave.open(filename, 'rb') as w:
        rate = w.getframerate()
        frames = w.getnframes()
        buffer = w.readframes(frames)

    return buffer, rate

# 1. Mozilla DeepSpeech

In [ ]:
#installing DeepSpeech

!pip install deepspeech==0.6.1

In [ ]:
import deepspeech

In [ ]:
# loading the model and unzipping it 
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.6.1/deepspeech-0.6.1-models.tar.gz
!tar -xvzf deepspeech-0.6.1-models.tar.gz
!ls -l ./deepspeech-0.6.1-models/

In [ ]:
# define the deepspeech model
model_file_path = 'deepspeech-0.6.1-models/output_graph.pbmm'
beam_width = 100
model = deepspeech.Model(model_file_path, beam_width)

lm_file_path = 'deepspeech-0.6.1-models/lm.binary'
trie_file_path = 'deepspeech-0.6.1-models/trie'
lm_alpha = 0.75
lm_beta = 1.85
model.enableDecoderWithLM(lm_file_path, trie_file_path, lm_alpha, lm_beta)

In [ ]:
#Defining the function for the model prediction


def Deep_Speech_stt(inp):

    
  file_list = []
  text_list = []


  for file in os.listdir(inp):
    try:
      current_file = os.path.join(inp, file)
      w = wave.open(current_file, 'r')
      rate = w.getframerate()
      frames = w.getnframes()
      buffer = w.readframes(frames)
      data16 = np.frombuffer(buffer, dtype=np.int16)
      text = model.stt(data16)

    # you can unmute the file printing of you want to 
      print(file)
      print(text)
      
      print(len(file_list))
      file_list.append(file)
      text_list.append(text)
      
    except:
        pass
          


  label  = pd.DataFrame(data=text_list, columns=[ "transcript"])
  files = pd.DataFrame(data=file_list, columns=["file"])

  predicted_data = label.join(files)
  return predicted_data

In [ ]:
# sample on how to get prediction 

inp2 = "/home/jupyter/CRP/Data/audio/Cleaned Audio files"
Deep_speech_prediction = Deep_Speech_stt(inp2)

#removing .wav from the end of the file name
Deep_speech_prediction['file'] = Deep_speech_prediction['file'].str.slice(0,-4,1)

Deep_speech_prediction.to_csv('Deep_speech_prediction.csv', encoding='utf-8', index=False)

# 2. Google Speech-to-text
- Google has [speech-to-text](https://cloud.google.com/speech-to-text/docs) as one of the Google Cloud services. 


2.1 Setup:

**Upload Google Cloud Cred file**

  - Have Google Cloud creds stored in a file named gc-creds.json, and upload it by running following code cell. See https://developers.google.com/accounts/docs/application-default-credentials for more details.

  - This may reqire enabling third-party cookies. Check out https://colab.research.google.com/notebooks/io.ipynb for other alternatives.

  - See the below link to learn how to create G-account Json:
https://cloud.google.com/docs/authentication/getting-started

  - More details are available [here](https://cloud.google.com/docs/authentication/production#auth-cloud-implicit-python)

In [26]:
# !pip install google-cloud-speech

In [ ]:
# upload the json file from the local machine
# ------this only works with Google colab.------------ 

#-------------------------------------------------
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [27]:
# Set environment variable

#Before running the code, upload the crednetial Json file to cloud and update its path
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/jupyter/CRP/gc_cred.json'

!ls -l $GOOGLE_APPLICATION_CREDENTIALS

-rw-r--r-- 1 jupyter jupyter 2341 Mar  8 14:23 /home/jupyter/CRP/gc_cred.json


In [28]:
#defining the google Speech to text function:

from google.cloud import speech_v1
from google.cloud.speech_v1 import enums

def google_batch_stt(filename: str):
    lang='en-US'
    encoding= 'LINEAR16'

    buffer, rate = read_wav_file(filename)
    client = speech_v1.SpeechClient()

    config = {
        'language_code': lang,
        'sample_rate_hertz': rate,
        'encoding': enums.RecognitionConfig.AudioEncoding[encoding]
    }

    audio = {
        'content': buffer
    }

    response = client.recognize(config, audio)
    # For bigger audio file, the previous line can be replaced with following:
    # operation = client.long_running_recognize(config, audio)
    # response = operation.result()

    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        return alternative.transcript


# defining the function to process all the files in a folder:
def Google_stt(inp):
  file_list = []
  text_list = []


  for file in os.listdir(inp):
      try:
          current_file = os.path.join(inp, file)
          text = google_batch_stt(current_file)

          print(len(file_list))
          print(file)
          print(text)

          file_list.append(file)
          text_list.append(text)
      except:
        pass

  label  = pd.DataFrame(data=text_list, columns=[ "transcript"])
  files = pd.DataFrame(data=file_list, columns=["file"])
  predicted_data = label.join(files)
  return predicted_data

In [29]:

# here is an example of how to run the google API

inp2 = "/home/jupyter/CRP/Data/audio/test" # insert the path here
Google_prediction = Google_stt(inp2)

#removing .wav from the end of the file name
Google_prediction['file'] = Google_prediction['file'].str.slice(0,-4,1)

Google_prediction.to_csv('Google_prediction.csv', encoding='utf-8', index=False)

# Finding the word Error rates

In [6]:
# a function to import a csv files and get the WER as compared with the labelled_data datafame
def prepare_data(label, predict):
    
    labelled = pd.read_csv(label)
    df_traslated = pd.read_csv(predict)
    merged = pd.merge(left=df_traslated, right=labelled, how='left',left_on='file', right_on='file')
    
    # sometimes the model may not be able to predict some values and might return a empty list
    # so we replace the empty list by nan
    merged.replace('', np.nan, inplace=True)

    #dropping the rows with nan
    merged = merged.dropna()
    
    # creating the lists from the two dataframes
    
    predicted_list = merged['transcript_x'].tolist()
    labelled_list = merged['transcript_y'].tolist()
    
    print("done preparing data")
    
    return predicted_list, labelled_list

In [7]:
def wer_by_step(labelled_list, predicted_list, chunk_size):
    """ Calculates the WER as the weighted average of WERs of smaller pieces of text.
    
    Parameters
    ----------
    labelled_list : list of strings (each string is a sentence)
        List of the original sentences (true labels)
    predicted_list : list of strings (each string is a sentence)
        List of the predicted sentences by the Speech Recognition Model chosen (predicted labels)
    chunk_size : integer (>= 2)
        Number of sentences per chunks on which we perform the WER calculation
        The smaller the chunk_size, the fastest the WER will be computed.

    Returns
    -------
    result : float64
        Total WER
        
    """
    
    start_time = time.time()
    list_len = len(predicted_list)
    total_word_count = 0
    weighted_sum = 0
    
    print("Calculating WER step by step")
    
    for i in range(0,int(floor(list_len/chunk_size))):
        word_count = 0
        labelled_list_short = labelled_list[i*chunk_size:min((i+1)*chunk_size-1,list_len-1)]
        predicted_list_short = predicted_list[i*chunk_size:min((i+1)*chunk_size-1,list_len-1)]
        error = wer(labelled_list_short, predicted_list_short)
        
        for items in labelled_list_short:
            word_count += len(items.split())
            
        weighted_sum += error*word_count
        total_word_count += word_count    
        #print("The WER for step %s is %s with %s words " % (i, error, word_count))
        
    result = weighted_sum/total_word_count
    
    print("The total WER is %s " % result)
    print("--- %s seconds ---" % (time.time() - start_time))
    
#    return result

## 1. Finding the WER for Deep Speech

In [12]:
label = "Deep_speech_prediction.csv"
predict = "labelled.csv"

predicted_list, labelled_list = prepare_data(label, predict)
wer_by_step(labelled_list, predicted_list, 10)

done preparing data
Calculating WER step by step
The total WER is 0.0799205097211694 
--- 79.19299840927124 seconds ---


## 2. Finding the WER for Google STT

In [13]:
label = "Google_prediction.csv"
predict = "labelled.csv"

predicted_list, labelled_list = prepare_data(label, predict)
wer_by_step(labelled_list, predicted_list, 10)

done preparing data
Calculating WER step by step
The total WER is 0.12726113373755843 
--- 78.4888288974762 seconds ---


In [35]:
label = "Google_prediction_sc.csv"
predict = "labelled.csv"

predicted_list, labelled_list = prepare_data(label, predict)
wer_by_step(labelled_list, predicted_list, 10)

done preparing data
Calculating WER step by step
The total WER is 0.12975855435469466 
--- 78.07587790489197 seconds ---


## Ohh Google why so smart!

The reason for the lower WER for Google STT as compared to Mozilla Deep Speech is the patented technology by Google for processgin the transcribed text.

For example Google Speech-to-Text returns “Mr.” instead of “Mister” and “7” for “seven”. 

Since all abbreviations and numbers are written out in the labeled dataset, the WER is higher for Google Speech-to-Text as it is supposed to be. 


# Auxilliary - Microsoft Azure

In addition to the Deep Speech and Google STT models, we also tested the Microsoft Azure's cloud base off-the-sheve technology for speech to text. Since this is a paid service, we just used a demo version to create the pipeline and did not transcribed the entire Librispeech corpus.

More info:

Microsoft Azure [Speech Services](https://azure.microsoft.com/en-in/services/cognitive-services/speech-services/) have [Speech to Text](https://azure.microsoft.com/en-in/services/cognitive-services/speech-to-text/) service.

## Setting service credentials

You can enable Speech service and find credentials for your account at [Microsoft Azure portal](https://portal.azure.com/). 

Just add Microsoft.CognitiveServicesSpeechServices to your azure account

You can open a free account [here](https://azure.microsoft.com/en-in/free/ai/).

In [12]:
#Setup - Install azure speech package
!pip install azure-cognitiveservices-speech

     |████████████████████████████████| 3.6 MB 4.5 MB/s eta 0:00:01


In [1]:
# Set service credentials
AZURE_SPEECH_KEY = '' # your Azure Speech Key here
AZURE_SERVICE_REGION = '' # your azure serive region hee

In [5]:
inp =  "/home/jupyter/CRP/Data/audio/test"

In [14]:
#defining the MIcrosoft Azure Speech to text function:

import azure.cognitiveservices.speech as speechsdk
  

def azure_batch_stt(filename: str):
    lang='en-US'
    encoding= 'LINEAR16'
    
    speech_config = speechsdk.SpeechConfig(
        subscription=AZURE_SPEECH_KEY,
        region=AZURE_SERVICE_REGION
    )
    audio_input = speechsdk.AudioConfig(filename=filename)
    speech_recognizer = speechsdk.SpeechRecognizer(
        speech_config=speech_config,
        audio_config=audio_input
    )
    result = speech_recognizer.recognize_once()

    return result.text if result.reason == speechsdk.ResultReason.RecognizedSpeech else None


# defining the function to process all the files in a folder:
def azure_stt(inp):
  file_list = []
  text_list = []


  for file in os.listdir(inp):
      try:
          current_file = os.path.join(inp, file)
          text =  azure_batch_stt(current_file)

          print(len(file_list))
          print(file)
          print(text)

          file_list.append(file)
          text_list.append(text)
      except:
        pass

  label  = pd.DataFrame(data=text_list, columns=[ "transcript"])
  files = pd.DataFrame(data=file_list, columns=["file"])
  predicted_data = label.join(files)
  return predicted_data

In [15]:
# here is an example of how to run the microsoft azure API

inp2 = "/home/jupyter/CRP/Data/audio/test" # insert the path here
azure_prediction = azure_stt(inp2)

#removing .wav from the end of the file name
azure_prediction['file'] = azure_prediction['file'].str.slice(0,-4,1)

azure_prediction.to_csv('azure_prediction.csv', encoding='utf-8', index=False)

0
1272-128104-0001.wav
Nor is Mr. Quilters manner less interesting than his matter.
1
1272-128104-0005.wav
It is obviously unnecessary for us to point out how luminous these criticisms are, how delicate in expression.
2
1272-128104-0003.wav
He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover in it, but little of Rocky Ithaca.
3
1272-128104-0004.wav
Lynels pictures are a sort of up guards and Adam paintings, and Masons Exquisite. It'll Zoras National as a Jingle poem.
4
1272-128104-0000.wav
Mr quilter is the Apostle of the middle classes and we're glad to welcome his gospel.
5
1272-128104-0002.wav
He tells us that at this festive season of the year with Christmas and roast beef looming before us, similes drawn from eating in its results occur most readily to the mind.
